In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

last_scrape = '2023-11-27'

In [3]:
playURL = 'https://api-web.nhle.com/v1/gamecenter/2023020175/play-by-play'

In [4]:
shiftURL = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId=2023020175'

In [5]:
file_name = f"data/summary_statsS-2023-11-28.csv"
player_biosS = pd.read_csv(file_name, index_col=False)
file_name = f"data/summary_statsG-2023-11-28.csv"
player_biosG = pd.read_csv(file_name, index_col=False)
player_bios = pd.concat([player_biosS, player_biosG], axis=0)
player_bios

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,secondaryPosition,tertiaryPosition,FPP60,FPPG,missedGames,gamesRemaining,fantasyPointsRemain,saves,shutout,creaseShare
0,8476453,N. Kucherov,TBL,R,26961,21,15.0,21.0,18.0,98,...,W,F,10.03,3.58,0,60,214.80,NaN,NaN,NaN
1,8476468,J. Miller,VAN,C,26486,22,13.0,20.0,16.0,50,...,C,F,9.77,3.27,0,60,196.20,NaN,NaN,NaN
2,8480069,C. Makar,COL,D,30559,21,6.0,26.0,14.0,51,...,D,D,8.21,3.32,0,61,202.52,NaN,NaN,NaN
3,8477933,S. Reinhart,FLA,C,26057,21,15.0,12.0,10.0,55,...,C,F,9.10,3.14,0,61,191.54,NaN,NaN,NaN
4,8480800,Q. Hughes,VAN,D,32684,22,8.0,25.0,15.0,68,...,D,D,7.17,2.96,0,60,177.60,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,8481611,P. Kochetkov,CAR,G,22230,7,NaN,NaN,NaN,151,...,NaN,NaN,0.26,0.23,0,19,4.95,133.0,1.0,30.81
72,8481692,D. Wolf,CGY,G,3585,1,NaN,NaN,NaN,38,...,NaN,NaN,-1.21,-1.20,0,2,-3.25,34.0,0.0,4.49
73,8482221,D. Levi,BUF,G,29916,9,NaN,NaN,NaN,251,...,NaN,NaN,-0.60,-0.56,0,22,-13.68,220.0,0.0,37.91
74,8482411,H. Shepard,WSH,G,7200,2,NaN,NaN,NaN,59,...,NaN,NaN,4.40,4.40,0,7,31.03,54.0,0.0,11.02


In [6]:
shifts = requests.get(shiftURL).json()
shifts = shifts['data']

# pprint(len(shifts))

shifts_df = pd.DataFrame()

print(shifts[0])

# for shift in shifts:
# {'id': 13376958, 'detailCode': 0, 'duration': '00:38', 'endTime': '00:38', 'eventDescription': None, 
#  'eventDetails': None, 'eventNumber': 6, 'firstName': 'Alex', 'gameId': 2023020175, 'hexValue': '#111111', 
#  'lastName': 'Killorn', 'period': 1, 'playerId': 8473986, 'shiftNumber': 1, 'startTime': '00:00', 'teamAbbrev': 'ANA', 
#  'teamId': 24, 'teamName': 'Anaheim Ducks', 'typeCode': 517}

for shift in shifts:
    temp = pd.DataFrame([shift])
    shifts_df = pd.concat([shifts_df, temp], axis=0, ignore_index=True)
    
    
events = shifts_df.loc[~shifts_df['eventDescription'].isna()]
shifts_df = shifts_df.loc[shifts_df['eventDescription'].isna()]  

shifts_df    





{'id': 13376958, 'detailCode': 0, 'duration': '00:38', 'endTime': '00:38', 'eventDescription': None, 'eventDetails': None, 'eventNumber': 6, 'firstName': 'Alex', 'gameId': 2023020175, 'hexValue': '#111111', 'lastName': 'Killorn', 'period': 1, 'playerId': 8473986, 'shiftNumber': 1, 'startTime': '00:00', 'teamAbbrev': 'ANA', 'teamId': 24, 'teamName': 'Anaheim Ducks', 'typeCode': 517}


,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,lastName,period,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode
0,13376958,0,00:38,00:38,None,None,6,Alex,2023020175,#111111,Killorn,1,8473986,1,00:00,ANA,24,Anaheim Ducks,517
1,13376959,0,00:47,04:07,None,None,92,Alex,2023020175,#111111,Killorn,1,8473986,2,03:20,ANA,24,Anaheim Ducks,517
2,13377110,0,01:07,07:47,None,None,181,Alex,2023020175,#111111,Killorn,1,8473986,3,06:40,ANA,24,Anaheim Ducks,517
3,13377111,0,01:03,10:26,None,None,258,Alex,2023020175,#111111,Killorn,1,8473986,4,09:23,ANA,24,Anaheim Ducks,517
4,13377265,0,00:32,13:34,None,None,294,Alex,2023020175,#111111,Killorn,1,8473986,5,13:02,ANA,24,Anaheim Ducks,517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,13377907,0,00:33,01:38,None,None,752,Leo,2023020175,#111111,Carlsson,3,8484153,12,01:05,ANA,24,Anaheim Ducks,517
703,13377908,0,01:10,03:31,None,None,762,Leo,2023020175,#111111,Carlsson,3,8484153,13,02:21,ANA,24,Anaheim Ducks,517
704,13377962,0,00:07,06:59,None,None,806,Leo,2023020175,#111111,Carlsson,3,8484153,14,06:52,ANA,24,Anaheim Ducks,517
705,13377963,0,01:06,10:23,None,None,822,Leo,2023020175,#111111,Carlsson,3,8484153,15,09:17,ANA,24,Anaheim Ducks,517


# Going to add the player's position to each row for separation

In [7]:
for index, row in shifts_df.iterrows():
    pos = player_bios.loc[player_bios['playerId'] == row['playerId']]['tertiaryPosition'].iloc[0]
    shifts_df.at[index, 'position'] = pos
    
shifts_df

,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,lastName,period,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,position
0,13376958,0,00:38,00:38,None,None,6,Alex,2023020175,#111111,Killorn,1,8473986,1,00:00,ANA,24,Anaheim Ducks,517,F
1,13376959,0,00:47,04:07,None,None,92,Alex,2023020175,#111111,Killorn,1,8473986,2,03:20,ANA,24,Anaheim Ducks,517,F
2,13377110,0,01:07,07:47,None,None,181,Alex,2023020175,#111111,Killorn,1,8473986,3,06:40,ANA,24,Anaheim Ducks,517,F
3,13377111,0,01:03,10:26,None,None,258,Alex,2023020175,#111111,Killorn,1,8473986,4,09:23,ANA,24,Anaheim Ducks,517,F
4,13377265,0,00:32,13:34,None,None,294,Alex,2023020175,#111111,Killorn,1,8473986,5,13:02,ANA,24,Anaheim Ducks,517,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,13377907,0,00:33,01:38,None,None,752,Leo,2023020175,#111111,Carlsson,3,8484153,12,01:05,ANA,24,Anaheim Ducks,517,F
703,13377908,0,01:10,03:31,None,None,762,Leo,2023020175,#111111,Carlsson,3,8484153,13,02:21,ANA,24,Anaheim Ducks,517,F
704,13377962,0,00:07,06:59,None,None,806,Leo,2023020175,#111111,Carlsson,3,8484153,14,06:52,ANA,24,Anaheim Ducks,517,F
705,13377963,0,01:06,10:23,None,None,822,Leo,2023020175,#111111,Carlsson,3,8484153,15,09:17,ANA,24,Anaheim Ducks,517,F


# Add integer seconds for ease of use

In [8]:
for index, row in shifts_df.iterrows():
    shifts_df.at[index, 'newStart'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['startTime'].split(':'))))
    shifts_df.at[index, 'newEnd'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['endTime'].split(':'))))
    
shifts_df['newStart'] = shifts_df['newStart'].astype(int)
shifts_df['newEnd'] = shifts_df['newEnd'].astype(int)

# Divide up the DF into teams, positions, periods

In [9]:
ducksF = shifts_df.loc[(shifts_df['teamAbbrev'] == 'ANA') & (shifts_df['position'] == 'F')]
ducksD = shifts_df.loc[(shifts_df['teamAbbrev'] == 'ANA') & (shifts_df['position'] == 'D')]
ducksG = shifts_df.loc[(shifts_df['teamAbbrev'] == 'ANA') & (shifts_df['position'] == 'G')]

knightsF = shifts_df.loc[(shifts_df['teamAbbrev'] == 'VGK') & (shifts_df['position'] == 'F')]
knightsD = shifts_df.loc[(shifts_df['teamAbbrev'] == 'VGK') & (shifts_df['position'] == 'D')]
knightsG = shifts_df.loc[(shifts_df['teamAbbrev'] == 'VGK') & (shifts_df['position'] == 'G')]

ducksF_1 = ducksF.loc[ducksF['period'] == 1]
ducksF_2 = ducksF.loc[ducksF['period'] == 2]
ducksF_3 = ducksF.loc[ducksF['period'] == 3]
ducksF_4 = ducksF.loc[ducksF['period'] == 4]

knightsF_1 = knightsF.loc[knightsF['period'] == 1]
knightsF_2 = knightsF.loc[knightsF['period'] == 2]
knightsF_3 = knightsF.loc[knightsF['period'] == 3]
knightsF_4 = knightsF.loc[knightsF['period'] == 4]

ducksD_1 = ducksD.loc[ducksD['period'] == 1]
ducksD_2 = ducksD.loc[ducksD['period'] == 2]
ducksD_3 = ducksD.loc[ducksD['period'] == 3]
ducksD_4 = ducksD.loc[ducksD['period'] == 4]

knightsD_1 = knightsD.loc[knightsD['period'] == 1]
knightsD_2 = knightsD.loc[knightsD['period'] == 2]
knightsD_3 = knightsD.loc[knightsD['period'] == 3]
knightsD_4 = knightsD.loc[knightsD['period'] == 4]

# Function that returns table of ice-time

In [103]:
def line_table(df):
    onice = pd.DataFrame()

    for i in range(1200):
        t = i+1
        activeName = []
        activeId = []
        for index, row in df.iterrows():
            if (t > row['newStart']) & (t <=row['newEnd']):
                activeName.append(f"{row['firstName']} {row['lastName']}")
                activeId.append(row['playerId'])
        activeName += [''] * (6 - len(activeName))
        activeId += [''] * (6 - len(activeId))
        temp = {f'p{k+1}': activeName[k] for k in range(6)}
        temp2 = {f'ID{k+1}': activeId[k] for k in range(6)}
        temp.update(temp2)
        temp['second'] = t

        temp['line'] = (temp['p1'] + '+' + temp['p2'] + '+' + temp['p3'] + '+' + temp['p4'] + '+' + temp['p5'] + '+' + temp['p6']).rstrip('+')
        temp['lineId'] = (str(temp['ID1']) + '+' + str(temp['ID2']) + '+' + str(temp['ID3']) + '+' + str(temp['ID4']) + '+' + str(temp['ID5']) + '+' + str(temp['ID6'])).rstrip('+')
        temp = pd.DataFrame([temp])

        onice = pd.concat([onice, temp], axis=0, ignore_index=True)
        

                          
    return onice

# Line table

In [104]:
line_table(ducksD_1)

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,ID5,ID6,second,line,lineId
0,Radko Gudas,Urho Vaakanainen,,,,,8475462,8480001,,,,,1,Radko Gudas+Urho Vaakanainen,8475462+8480001
1,Radko Gudas,Urho Vaakanainen,,,,,8475462,8480001,,,,,2,Radko Gudas+Urho Vaakanainen,8475462+8480001
2,Radko Gudas,Urho Vaakanainen,,,,,8475462,8480001,,,,,3,Radko Gudas+Urho Vaakanainen,8475462+8480001
3,Radko Gudas,Urho Vaakanainen,,,,,8475462,8480001,,,,,4,Radko Gudas+Urho Vaakanainen,8475462+8480001
4,Radko Gudas,Urho Vaakanainen,,,,,8475462,8480001,,,,,5,Radko Gudas+Urho Vaakanainen,8475462+8480001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,Radko Gudas,Urho Vaakanainen,,,,,8475462,8480001,,,,,1196,Radko Gudas+Urho Vaakanainen,8475462+8480001
1196,Radko Gudas,Urho Vaakanainen,,,,,8475462,8480001,,,,,1197,Radko Gudas+Urho Vaakanainen,8475462+8480001
1197,Radko Gudas,Urho Vaakanainen,,,,,8475462,8480001,,,,,1198,Radko Gudas+Urho Vaakanainen,8475462+8480001
1198,Radko Gudas,Urho Vaakanainen,,,,,8475462,8480001,,,,,1199,Radko Gudas+Urho Vaakanainen,8475462+8480001


In [105]:
line_table(ducksD_1)['line'].value_counts()

Cam Fowler+Jackson LaCombe                      370
Ilya Lyubushkin+Pavel Mintyukov                 337
Radko Gudas+Urho Vaakanainen                    311
Radko Gudas+Cam Fowler                           43
Radko Gudas+Jackson LaCombe                      35
Radko Gudas+Pavel Mintyukov                      35
Ilya Lyubushkin+Jackson LaCombe                  20
Jackson LaCombe+Pavel Mintyukov                  17
Cam Fowler+Pavel Mintyukov                       12
Radko Gudas+Ilya Lyubushkin                      11
Urho Vaakanainen+Ilya Lyubushkin                  6
Radko Gudas+Urho Vaakanainen+Jackson LaCombe      1
Cam Fowler+Ilya Lyubushkin                        1
Cam Fowler+Ilya Lyubushkin+Pavel Mintyukov        1
Name: line, dtype: int64

# Plays

In [107]:
plays = requests.get(playURL).json()
plays = plays['plays']

plays = pd.DataFrame(plays)

plays



for index, row in plays.iterrows():
    if row['typeDescKey'] == 'faceoff':
        plays.at[index, 'second'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['timeInPeriod'].split(':'))))+1
    else:
        plays.at[index, 'second'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['timeInPeriod'].split(':'))))

plays['second'] = plays['second'].astype(int)

first_plays = plays.loc[plays['period'] == 1]


KeyError: 'faceoff'

# Extract the dictionaries

In [108]:
def extract_values(row, key):
    return row.get(key)

keys_to_extract1 = ['periodType']

for key in keys_to_extract1:
    plays[key] = plays['periodDescriptor'].apply(lambda x: extract_values(x, key))
    
keys_to_extract_faceoff = ['eventOwnerTeamId', 'losingPlayerId', 'winningPlayerId', 'xCoord', 'yCoord', 'zoneCode']
keys_to_extract_takeaway = ['eventOwnerTeamId', 'playerId', 'xCoord', 'yCoord', 'zoneCode']
keys_to_extract_shot = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'shootingPlayerId', 'goalieInNetId', 'awaySOG', 'homeSOG']
keys_to_extract_hit = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode',  'hittingPlayerId', 'hitteePlayerId']
keys_to_extract_missedshot = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'shootingPlayerId', 'goalieInNetId', 'reason']
keys_to_extract_block = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode',  'blockingPlayerId', 'shootingPlayerId']
keys_to_extract_giveaway = ['eventOwnerTeamId', 'playerId', 'xCoord', 'yCoord', 'zoneCode']
keys_to_extract_stoppage = ['reason']
keys_to_extract_goal = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'scoringPlayerId', 'scoringPlayerTotal', 'assist1PlayerId', 'assist1PlayerTotal', 'assist2PlayerId', 'assist2PlayerTotal', 'goalieInNetId', 'awayScore', 'homeScore']
keys_to_extract_penalty = ['eventOwnerTeamId', 'committedByPlayerId', 'xCoord', 'yCoord', 'zoneCode', 'typeCode', 'descKey', 'duration', 'drawnByPlayerId']

keys_lists = [
    keys_to_extract_faceoff,
    keys_to_extract_takeaway,
    keys_to_extract_shot,
    keys_to_extract_hit,
    keys_to_extract_missedshot,
    keys_to_extract_block,
    keys_to_extract_giveaway,
    keys_to_extract_stoppage,
    keys_to_extract_goal,
    keys_to_extract_penalty
]

plays

,eventId,period,periodDescriptor,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,details,second,periodType
0,102,1,"{'number': 1, 'periodType': 'REG'}",00:00,20:00,1551,left,520,period-start,8,NaN,0,REG
1,101,1,"{'number': 1, 'periodType': 'REG'}",00:00,20:00,1551,left,502,faceoff,11,"{'eventOwnerTeamId': 54, 'losingPlayerId': 847...",1,REG
2,103,1,"{'number': 1, 'periodType': 'REG'}",00:27,19:33,1551,left,525,takeaway,12,"{'xCoord': -14, 'yCoord': 39, 'zoneCode': 'N',...",27,REG
3,65,1,"{'number': 1, 'periodType': 'REG'}",01:13,18:47,1551,left,506,shot-on-goal,23,"{'xCoord': -64, 'yCoord': 17, 'zoneCode': 'O',...",73,REG
4,66,1,"{'number': 1, 'periodType': 'REG'}",01:23,18:37,1551,left,506,shot-on-goal,24,"{'xCoord': -67, 'yCoord': 28, 'zoneCode': 'O',...",83,REG
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,975,3,"{'number': 3, 'periodType': 'REG'}",19:22,00:38,1551,left,506,shot-on-goal,737,"{'xCoord': -57, 'yCoord': -32, 'zoneCode': 'O'...",1162,REG
257,374,3,"{'number': 3, 'periodType': 'REG'}",19:46,00:14,1551,left,516,stoppage,739,{'reason': 'puck-in-crowd'},1186,REG
258,575,3,"{'number': 3, 'periodType': 'REG'}",19:46,00:14,1551,left,502,faceoff,742,"{'eventOwnerTeamId': 24, 'losingPlayerId': 847...",1187,REG
259,375,3,"{'number': 3, 'periodType': 'REG'}",20:00,00:00,1551,left,521,period-end,743,NaN,1200,REG


In [109]:
def extract_values(row, keys):
    if isinstance(row, dict):
        return {key: row.get(key) for key in keys}
    else:
        return {key: None for key in keys}

for keys_to_extract in keys_lists:
    # Apply the function to each row
    extracted_df = pd.DataFrame(plays['details'].apply(lambda x: extract_values(x, keys_to_extract)).tolist())
    plays = pd.concat([plays, extracted_df], axis=1)
    
plays = plays.groupby(plays.columns, axis=1).first()
plays = plays.drop(columns=['details', 'periodDescriptor'])

# Convert to integers

In [110]:
plays.columns.tolist()
convert_columns = ['assist1PlayerId', 'assist2PlayerId', 'blockingPlayerId', 'committedByPlayerId', 'drawnByPlayerId', 'goalieInNetId', 'hitteePlayerId', 'hittingPlayerId', 'losingPlayerId', 'playerId', 'scoringPlayerId', 'shootingPlayerId', 'winningPlayerId']

plays[convert_columns] = plays[convert_columns].astype(pd.Int64Dtype(), errors='ignore')

convert_columns2 = ['assist1PlayerTotal', 'assist2PlayerTotal', 'awaySOG', 'awayScore', 'homeSOG', 'homeScore', 'scoringPlayerTotal', 'eventOwnerTeamId', 'duration']
plays[convert_columns2] = plays[convert_columns2].astype(pd.Int64Dtype(), errors='ignore')


# Add one second to faceoff


In [30]:
# Careful! Only run once
# plays.loc[plays['typeDescKey'] == 'faceoff', 'second'] += 1

# Divide by period


In [111]:
plays_1 = plays.loc[plays['period'] == 1]
plays_2 = plays.loc[plays['period'] == 2]
plays_3 = plays.loc[plays['period'] == 3]
plays_4 = plays.loc[plays['period'] == 4]
plays_1


,assist1PlayerId,assist1PlayerTotal,assist2PlayerId,assist2PlayerTotal,awaySOG,awayScore,blockingPlayerId,committedByPlayerId,descKey,drawnByPlayerId,...,situationCode,sortOrder,timeInPeriod,timeRemaining,typeCode,typeDescKey,winningPlayerId,xCoord,yCoord,zoneCode
0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,8,00:00,20:00,520,period-start,<NA>,NaN,NaN,None
1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,11,00:00,20:00,502,faceoff,8476448,0.0,0.0,N
2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,12,00:27,19:33,525,takeaway,<NA>,-14.0,39.0,N
3,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,None,<NA>,...,1551,23,01:13,18:47,506,shot-on-goal,<NA>,-64.0,17.0,O
4,<NA>,<NA>,<NA>,<NA>,2,<NA>,<NA>,<NA>,None,<NA>,...,1551,24,01:23,18:37,506,shot-on-goal,<NA>,-67.0,28.0,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,<NA>,<NA>,<NA>,<NA>,9,<NA>,<NA>,<NA>,None,<NA>,...,1551,257,18:57,01:03,506,shot-on-goal,<NA>,86.0,7.0,O
71,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,266,19:36,00:24,503,hit,<NA>,97.0,-6.0,D
72,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,267,19:39,00:21,503,hit,<NA>,74.0,35.0,D
73,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,8481167,<NA>,None,<NA>,...,1551,269,19:43,00:17,508,blocked-shot,<NA>,70.0,-3.0,D


In [112]:
types = plays['typeDescKey'].unique()
types

array(['period-start', 'faceoff', 'takeaway', 'shot-on-goal', 'hit',
       'missed-shot', 'blocked-shot', 'giveaway', 'stoppage', 'goal',
       'penalty', 'period-end', 'game-end'], dtype=object)

# Attempt to append to shifts

In [66]:
ducksFshifts_1 = line_table(ducksF_1)
ducksDshifts_1 = line_table(ducksD_1)


knightsFshifts_1 = line_table(knightsF_1)
knightsDshifts_1 = line_table(knightsD_1)


ducksFshifts_2 = line_table(ducksF_2)
ducksDshifts_2 = line_table(ducksD_2)


knightsFshifts_2 = line_table(knightsF_2)
knightsDshifts_2 = line_table(knightsD_2)


ducksFshifts_3 = line_table(ducksF_3)
ducksDshifts_3 = line_table(ducksD_3)


knightsFshifts_3 = line_table(knightsF_3)
knightsDshifts_3 = line_table(knightsD_3)


ducksFshifts_4 = line_table(ducksF_4)
ducksDshifts_4 = line_table(ducksD_4)


knightsFshifts_4 = line_table(knightsF_4)
knightsDshifts_4 = line_table(knightsD_4)


knightsDshifts_3

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,ID5,ID6,second,line
0,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,,,1,Alex Pietrangelo+Brayden McNabb
1,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,,,2,Alex Pietrangelo+Brayden McNabb
2,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,,,3,Alex Pietrangelo+Brayden McNabb
3,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,,,4,Alex Pietrangelo+Brayden McNabb
4,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,,,5,Alex Pietrangelo+Brayden McNabb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,Brayden McNabb,Brayden Pachal,,,,,8475188,8481167,,,,,1196,Brayden McNabb+Brayden Pachal
1196,Brayden McNabb,Brayden Pachal,,,,,8475188,8481167,,,,,1197,Brayden McNabb+Brayden Pachal
1197,Brayden McNabb,Brayden Pachal,,,,,8475188,8481167,,,,,1198,Brayden McNabb+Brayden Pachal
1198,Brayden McNabb,Brayden Pachal,,,,,8475188,8481167,,,,,1199,Brayden McNabb+Brayden Pachal


In [67]:
ducksFshiftsPlays_1 = pd.merge(ducksFshifts_1, plays_1, how='left', on='second')
ducksFshiftsPlays_2 = pd.merge(ducksFshifts_2, plays_2, how='left', on='second')
ducksFshiftsPlays_3 = pd.merge(ducksFshifts_3, plays_3, how='left', on='second')
ducksFshiftsPlays_4 = pd.merge(ducksFshifts_4, plays_4, how='left', on='second')

ducksDshiftsPlays_1 = pd.merge(ducksDshifts_1, plays_1, how='left', on='second')
ducksDshiftsPlays_2 = pd.merge(ducksDshifts_2, plays_2, how='left', on='second')
ducksDshiftsPlays_3 = pd.merge(ducksDshifts_3, plays_3, how='left', on='second')
ducksDshiftsPlays_4 = pd.merge(ducksDshifts_4, plays_4, how='left', on='second')

knightsFshiftsPlays_1 = pd.merge(knightsFshifts_1, plays_1, how='left', on='second')
knightsFshiftsPlays_2 = pd.merge(knightsFshifts_2, plays_2, how='left', on='second')
knightsFshiftsPlays_3 = pd.merge(knightsFshifts_3, plays_3, how='left', on='second')
knightsFshiftsPlays_4 = pd.merge(knightsFshifts_4, plays_4, how='left', on='second')

knightsDshiftsPlays_1 = pd.merge(knightsDshifts_1, plays_1, how='left', on='second')
knightsDshiftsPlays_2 = pd.merge(knightsDshifts_2, plays_2, how='left', on='second')
knightsDshiftsPlays_3 = pd.merge(knightsDshifts_3, plays_3, how='left', on='second')
knightsDshiftsPlays_4 = pd.merge(knightsDshifts_4, plays_4, how='left', on='second')

knightsDshiftsPlays_3

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,...,situationCode,sortOrder,timeInPeriod,timeRemaining,typeCode,typeDescKey,winningPlayerId,xCoord,yCoord,zoneCode
0,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,...,1451,529.0,00:00,20:00,502.0,faceoff,8482745,0.0,0.0,N
1,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
2,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
3,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
4,Alex Pietrangelo,Brayden McNabb,,,,,8474565,8475188,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,Brayden McNabb,Brayden Pachal,,,,,8475188,8481167,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1198,Brayden McNabb,Brayden Pachal,,,,,8475188,8481167,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1199,Brayden McNabb,Brayden Pachal,,,,,8475188,8481167,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1200,Brayden McNabb,Brayden Pachal,,,,,8475188,8481167,,,...,1551,743.0,20:00,00:00,521.0,period-end,<NA>,NaN,NaN,NaN


# Add a period marker

In [68]:
ducksFshiftsPlays_1['period'] = 1
ducksFshiftsPlays_2['period'] = 2
ducksFshiftsPlays_3['period'] = 3 
ducksFshiftsPlays_4['period'] = 4

ducksDshiftsPlays_1['period'] = 1
ducksDshiftsPlays_2['period'] = 2 
ducksDshiftsPlays_3['period'] = 3  
ducksDshiftsPlays_4['period'] = 4

knightsFshiftsPlays_1['period'] = 1
knightsFshiftsPlays_2['period'] = 2 
knightsFshiftsPlays_3['period'] = 3  
knightsFshiftsPlays_4['period'] = 4 

knightsDshiftsPlays_1['period'] = 1 
knightsDshiftsPlays_2['period'] = 2 
knightsDshiftsPlays_3['period'] = 3  
knightsDshiftsPlays_4['period'] = 4 

# Attemtp to find if PP = True

In [113]:

ANApp = 0
VGKpp = 0
for index, row in ducksFshiftsPlays_1.iterrows():
    if row['typeDescKey'] == 'penalty':
        if row['eventOwnerTeamId'] == 24:
            VGKpp = 1
            pen_time = row['second']

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,...,situationCode,sortOrder,timeInPeriod,timeRemaining,typeCode,typeDescKey,winningPlayerId,xCoord,yCoord,zoneCode
0,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,...,1551,11.0,00:00,20:00,502.0,faceoff,8476448,0.0,0.0,N
1,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
2,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
3,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
4,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1198,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1199,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1200,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN


In [123]:
pp_finder = shifts_df.copy()

for index, row in pp_finder.iterrows():
    if row['period'] == 2:
        pp_finder.at[index, 'newStart'] = row['newStart'] + 1200
        pp_finder.at[index, 'newEnd'] = row['newEnd'] + 1200
    if row['period'] == 3:
        pp_finder.at[index, 'newStart'] = row['newStart'] + 2400
        pp_finder.at[index, 'newEnd'] = row['newEnd'] + 2400
    if row['period'] == 4:
        pp_finder.at[index, 'newStart'] = row['newStart'] + 3600
        pp_finder.at[index, 'newEnd'] = row['newEnd'] + 3600
        
pp_finder = pp_finder.loc[(pp_finder['position'] == 'F') | (pp_finder['position'] == 'D')]

pp_finder

,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,...,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,position,newStart,newEnd
0,13376958,0,00:38,00:38,None,None,6,Alex,2023020175,#111111,...,8473986,1,00:00,ANA,24,Anaheim Ducks,517,F,0,38
1,13376959,0,00:47,04:07,None,None,92,Alex,2023020175,#111111,...,8473986,2,03:20,ANA,24,Anaheim Ducks,517,F,200,247
2,13377110,0,01:07,07:47,None,None,181,Alex,2023020175,#111111,...,8473986,3,06:40,ANA,24,Anaheim Ducks,517,F,400,467
3,13377111,0,01:03,10:26,None,None,258,Alex,2023020175,#111111,...,8473986,4,09:23,ANA,24,Anaheim Ducks,517,F,563,626
4,13377265,0,00:32,13:34,None,None,294,Alex,2023020175,#111111,...,8473986,5,13:02,ANA,24,Anaheim Ducks,517,F,782,814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,13377907,0,00:33,01:38,None,None,752,Leo,2023020175,#111111,...,8484153,12,01:05,ANA,24,Anaheim Ducks,517,F,2465,2498
703,13377908,0,01:10,03:31,None,None,762,Leo,2023020175,#111111,...,8484153,13,02:21,ANA,24,Anaheim Ducks,517,F,2541,2611
704,13377962,0,00:07,06:59,None,None,806,Leo,2023020175,#111111,...,8484153,14,06:52,ANA,24,Anaheim Ducks,517,F,2812,2819
705,13377963,0,01:06,10:23,None,None,822,Leo,2023020175,#111111,...,8484153,15,09:17,ANA,24,Anaheim Ducks,517,F,2957,3023


In [129]:

onice = pd.DataFrame()

for i in range(4800):
    t = i+1
    activeName = []
    activeId = []
    activeTeam = []
    for index, row in pp_finder.iterrows():
        if (t > row['newStart']) & (t <=row['newEnd']):
#             activeName.append(f"{row['firstName']} {row['lastName']}")
            activeId.append(row['playerId'])
            activeTeam.append(row['teamAbbrev'])
        
#     activeName += [''] * (12 - len(activeName))
    activeId += [''] * (12 - len(activeId))
    activeTeam += [''] * (12 - len(activeTeam))
#     temp = {f'p{k+1}': activeName[k] for k in range(12)}
    temp = {f'ID{k+1}': activeTeam[k] for k in range(12)}
#     temp.update(temp2)
    temp['second'] = t

#     temp['line'] = (temp['p1'] + '+' + temp['p2'] + '+' + temp['p3'] + '+' + temp['p4'] + '+' + temp['p5'] + '+' + temp['p6']).rstrip('+')
#     temp['lineId'] = (str(temp['ID1']) + '+' + str(temp['ID2']) + '+' + str(temp['ID3']) + '+' + str(temp['ID4']) + '+' + str(temp['ID5']) + '+' + str(temp['ID6'])).rstrip('+')
    temp = pd.DataFrame([temp])

    onice = pd.concat([onice, temp], axis=0, ignore_index=True)
        

                          


In [136]:
for index, row in onice.iterrows():
    away = 0
    home = 0
    for k in range(10):
        p = k+1
        if row[f"ID{p}"] == 'ANA':
            home +=1
        elif row[f"ID{p}"] == 'VGK':
            away +=1
    onice.at[index, 'ANA_players'] = home
    onice.at[index, 'VGK_players'] = away
    
    if home > away:
        onice.at[index, 'ANA_pp'] = 1
        onice.at[index, 'VGK_pp'] = 0
    elif home < away:
        onice.at[index, 'ANA_pp'] = 0
        onice.at[index, 'VGK_pp'] = 1
    else:
        onice.at[index, 'ANA_pp'] = 0
        onice.at[index, 'VGK_pp'] = 0
onice    

,ID1,ID2,ID3,ID4,ID5,ID6,ID7,ID8,ID9,ID10,ID11,ID12,second,ANA_players,VGK_players,ANA_pp,VGK_pp
0,ANA,ANA,ANA,VGK,ANA,VGK,VGK,VGK,ANA,VGK,,,1,5.0,5.0,0.0,0.0
1,ANA,ANA,ANA,VGK,ANA,VGK,VGK,VGK,ANA,VGK,,,2,5.0,5.0,0.0,0.0
2,ANA,ANA,ANA,VGK,ANA,VGK,VGK,VGK,ANA,VGK,,,3,5.0,5.0,0.0,0.0
3,ANA,ANA,ANA,VGK,ANA,VGK,VGK,VGK,ANA,VGK,,,4,5.0,5.0,0.0,0.0
4,ANA,ANA,ANA,VGK,ANA,VGK,VGK,VGK,ANA,VGK,,,5,5.0,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,,,,,,,,,,,,,4796,0.0,0.0,0.0,0.0
4796,,,,,,,,,,,,,4797,0.0,0.0,0.0,0.0
4797,,,,,,,,,,,,,4798,0.0,0.0,0.0,0.0
4798,,,,,,,,,,,,,4799,0.0,0.0,0.0,0.0


In [135]:
onice['VGK_players'].value_counts()

5.0    3226
0.0    1200
4.0     352
6.0      22
Name: VGK_players, dtype: int64

In [98]:
ducksFshiftsPlays = pd.concat([ducksFshiftsPlays_1, ducksFshiftsPlays_2, ducksFshiftsPlays_3, ducksFshiftsPlays_4], axis=0)

ducksDshiftsPlays = pd.concat([ducksDshiftsPlays_1, ducksDshiftsPlays_2, ducksDshiftsPlays_3, ducksDshiftsPlays_4], axis=0)

knightsFshiftsPlays = pd.concat([knightsFshiftsPlays_1, knightsFshiftsPlays_2, knightsFshiftsPlays_3, knightsFshiftsPlays_4], axis=0)

knightsDshiftsPlays = pd.concat([knightsDshiftsPlays_1, knightsDshiftsPlays_2, knightsDshiftsPlays_3, knightsDshiftsPlays_4], axis=0)

ducksFshiftsPlays['period'].value_counts()

1    1202
2    1202
3    1202
4    1200
Name: period, dtype: int64

In [70]:
column_names = ducksFshiftsPlays.columns.to_list()
print(list(x for x in column_names))

['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'ID1', 'ID2', 'ID3', 'ID4', 'ID5', 'ID6', 'second', 'line', 'assist1PlayerId', 'assist1PlayerTotal', 'assist2PlayerId', 'assist2PlayerTotal', 'awaySOG', 'awayScore', 'blockingPlayerId', 'committedByPlayerId', 'descKey', 'drawnByPlayerId', 'duration', 'eventId', 'eventOwnerTeamId', 'goalieInNetId', 'hitteePlayerId', 'hittingPlayerId', 'homeSOG', 'homeScore', 'homeTeamDefendingSide', 'losingPlayerId', 'period', 'periodType', 'playerId', 'reason', 'scoringPlayerId', 'scoringPlayerTotal', 'shootingPlayerId', 'shotType', 'situationCode', 'sortOrder', 'timeInPeriod', 'timeRemaining', 'typeCode', 'typeDescKey', 'winningPlayerId', 'xCoord', 'yCoord', 'zoneCode']


In [84]:

# Temp DF used to eliminate some doubled seconds for plays that occured in same moment

line_timing_temp = ducksFshiftsPlays.copy()

line_timing_temp = line_timing_temp.reset_index()

for index, row in line_timing_temp.iterrows():
    line_timing_temp.at[index, 'uSecond'] = f"{row['period']}-{row['second']}"
    

df_no_duplicates = line_timing_temp.drop_duplicates(subset='uSecond')

df_no_duplicates['line'].value_counts()



                                                1200
Alex Killorn+Adam Henrique+Jakob Silfverberg     695
Ryan Strome+Frank Vatrano+Mason McTavish         601
Troy Terry+Trevor Zegras+Leo Carlsson            562
Sam Carrick+Ross Johnston+Brett Leason           245
                                                ... 
Sam Carrick+Ryan Strome+Ross Johnston              1
Adam Henrique+Jakob Silfverberg                    1
Ryan Strome+Ross Johnston                          1
Troy Terry+Mason McTavish+Leo Carlsson             1
Adam Henrique+Ryan Strome                          1
Name: line, Length: 77, dtype: int64

In [85]:
ducksFshiftsPlays.loc[(ducksFshiftsPlays['line'] == 'Sam Carrick+Ross Johnston+Brett Leason') & (ducksFshiftsPlays['eventOwnerTeamId'] == 24)]['typeDescKey'].value_counts()

hit             5
blocked-shot    4
giveaway        3
missed-shot     1
faceoff         1
goal            1
Name: typeDescKey, dtype: int64

In [97]:
# array(['period-start', 'faceoff', 'takeaway', 'shot-on-goal', 'hit',
#        'missed-shot', 'blocked-shot', 'giveaway', 'stoppage', 'goal',
#        'penalty', 'period-end', 'game-end'], dtype=object)



# keys_to_extract_faceoff = ['eventOwnerTeamId', 'losingPlayerId', 'winningPlayerId', 'xCoord', 'yCoord', 'zoneCode']
# keys_to_extract_takeaway = ['eventOwnerTeamId', 'playerId', 'xCoord', 'yCoord', 'zoneCode']
# keys_to_extract_shot = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'shootingPlayerId', 'goalieInNetId', 'awaySOG', 'homeSOG']
# keys_to_extract_hit = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode',  'hittingPlayerId', 'hitteePlayerId']
# keys_to_extract_missedshot = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'shootingPlayerId', 'goalieInNetId', 'reason']
# keys_to_extract_block = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode',  'blockingPlayerId', 'shootingPlayerId']
# keys_to_extract_giveaway = ['eventOwnerTeamId', 'playerId', 'xCoord', 'yCoord', 'zoneCode']
# keys_to_extract_stoppage = ['reason']
# keys_to_extract_goal = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'scoringPlayerId', 'scoringPlayerTotal', 'assist1PlayerId', 'assist1PlayerTotal', 'assist2PlayerId', 'assist2PlayerTotal', 'goalieInNetId', 'awayScore', 'homeScore']
# keys_to_extract_penalty = ['eventOwnerTeamId', 'committedByPlayerId', 'xCoord', 'yCoord', 'zoneCode', 'typeCode', 'descKey', 'duration', 'drawnByPlayerId']


# FACEOFFS -- eventOwnerTeamId == team that won faceoff
# BLOCKED SHOTS -- eventOwnerTeamId == team blocked the shot (opposite of Corsi)
# HITS -- eventOwnerTeamId == team that threw hit

faceoff = plays.loc[plays['typeDescKey'] == 'hit']
for index, row in faceoff.iterrows():
    print(
        f"{row['eventOwnerTeamId']} " 
        f"{player_bios.loc[player_bios['playerId'] == row['hittingPlayerId']]['name'].iloc[0]} laid the smack down on "
        f"{player_bios.loc[player_bios['playerId'] == row['hitteePlayerId']]['name'].iloc[0]}"
    )

54 W. Carrier laid the smack down on I. Lyubushkin
24 S. Carrick laid the smack down on B. McNabb
24 J. Silfverberg laid the smack down on A. Pietrangelo
24 R. Gudas laid the smack down on M. Amadio
24 J. LaCombe laid the smack down on J. Eichel
54 B. Howden laid the smack down on R. Gudas
54 P. Cotter laid the smack down on A. Killorn
54 B. Howden laid the smack down on S. Carrick
24 R. Johnston laid the smack down on P. Cotter
24 R. Johnston laid the smack down on B. Pachal
24 C. Fowler laid the smack down on M. Amadio
54 I. Barbashev laid the smack down on C. Fowler
24 J. LaCombe laid the smack down on K. Korczak
24 S. Carrick laid the smack down on K. Kolesar
54 W. Karlsson laid the smack down on R. Strome
54 B. Pachal laid the smack down on J. Silfverberg
54 W. Carrier laid the smack down on U. Vaakanainen
24 R. Johnston laid the smack down on B. Howden
54 K. Kolesar laid the smack down on B. Leason
54 B. Pachal laid the smack down on T. Zegras
54 K. Kolesar laid the smack down on

In [99]:
knightsDshiftsPlays

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,...,situationCode,sortOrder,timeInPeriod,timeRemaining,typeCode,typeDescKey,winningPlayerId,xCoord,yCoord,zoneCode
0,Brayden McNabb,Shea Theodore,,,,,8475188,8477447,,,...,1551,11.0,00:00,20:00,502.0,faceoff,8476448,0.0,0.0,N
1,Brayden McNabb,Shea Theodore,,,,,8475188,8477447,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
2,Brayden McNabb,Shea Theodore,,,,,8475188,8477447,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
3,Brayden McNabb,Shea Theodore,,,,,8475188,8477447,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
4,Brayden McNabb,Shea Theodore,,,,,8475188,8477447,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,,,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1196,,,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1197,,,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN
1198,,,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN


# Expore how to handle stoppages


In [338]:
ducksF_1.loc[ducksF_1['newEnd'] == 814]

,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,...,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,position,newStart,newEnd
4,13377265,0,00:32,13:34,None,None,294,Alex,2023020175,#111111,...,8473986,5,13:02,ANA,24,Anaheim Ducks,517,F,782,814
54,13377115,0,00:52,13:34,None,None,291,Adam,2023020175,#111111,...,8474641,5,12:42,ANA,24,Anaheim Ducks,517,F,762,814
78,13377276,0,00:52,13:34,None,None,291,Jakob,2023020175,#111111,...,8475164,5,12:42,ANA,24,Anaheim Ducks,517,F,762,814


In [339]:
ducksF_1.loc[ducksF_1['newStart'] == 814]

,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,...,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,position,newStart,newEnd
392,13377310,0,00:41,14:15,None,None,302,Frank,2023020175,#111111,...,8478366,5,13:34,ANA,24,Anaheim Ducks,517,F,814,855
654,13377337,0,00:47,14:21,None,None,302,Mason,2023020175,#111111,...,8482745,5,13:34,ANA,24,Anaheim Ducks,517,F,814,861


In [340]:
plays_1.loc[(plays_1['typeDescKey'] == 'stoppage') | (plays_1['typeDescKey'] == 'penalty') | (plays_1['typeDescKey'] == 'goal')]



,assist1PlayerId,assist1PlayerTotal,assist2PlayerId,assist2PlayerTotal,awaySOG,awayScore,blockingPlayerId,committedByPlayerId,descKey,drawnByPlayerId,...,situationCode,sortOrder,timeInPeriod,timeRemaining,typeCode,typeDescKey,winningPlayerId,xCoord,yCoord,zoneCode
20,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,103,06:59,13:01,None,stoppage,<NA>,NaN,NaN,None
27,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,118,08:12,11:48,None,stoppage,<NA>,NaN,NaN,None
29,8478403,8,<NA>,<NA>,<NA>,1,<NA>,<NA>,None,<NA>,...,1551,123,08:51,11:09,None,goal,<NA>,-85.0,-3.0,O
32,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,128,09:23,10:37,None,stoppage,<NA>,NaN,NaN,None
42,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,164,11:57,08:03,None,stoppage,<NA>,NaN,NaN,None
48,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,<NA>,...,1551,180,13:17,06:43,None,stoppage,<NA>,NaN,NaN,None
52,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,8474641,interference,8477478,...,1551,186,13:34,06:26,MIN,penalty,<NA>,-87.0,-1.0,D
53,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,8477478,tripping,8474641,...,1541,188,13:34,06:26,MIN,penalty,<NA>,-88.0,-1.0,O


In [341]:
duck_table_1 = line_table(ducksF_1)
duck_table_1.loc[duck_table_1['second'] == 814]

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,ID5,ID6,second,line
813,Alex Killorn,Adam Henrique,Jakob Silfverberg,,,,8473986,8474641,8475164,,,,814,Alex Killorn+Adam Henrique+Jakob Silfverberg


In [343]:
plays_1.loc[(plays_1['typeDescKey'] == 'faceoff') | (plays_1['typeDescKey'] == 'period-start')]['second']



0       0
1       0
21    419
28    492
30    531
33    563
43    717
49    797
54    814
Name: second, dtype: int64

In [344]:
ducksF_1.loc[ducksF_1['newStart'] == 814]

,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,...,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,position,newStart,newEnd
392,13377310,0,00:41,14:15,None,None,302,Frank,2023020175,#111111,...,8478366,5,13:34,ANA,24,Anaheim Ducks,517,F,814,855
654,13377337,0,00:47,14:21,None,None,302,Mason,2023020175,#111111,...,8482745,5,13:34,ANA,24,Anaheim Ducks,517,F,814,861


In [49]:
array_from_index = rows_with_trio.index.values
array_from_index

array([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,
         23,   24,   25,   26,   27,   28,   29,   30,   31,   32,   33,
         34,   35,   36,  200,  201,  202,  203,  204,  205,  206,  207,
        208,  209,  210,  211,  212,  213,  214,  215,  216,  217,  218,
        219,  220,  221,  222,  223,  224,  225,  226,  227,  228,  229,
        230,  231,  232,  233,  234,  235,  236,  237,  238,  239,  240,
        241,  242,  243,  244,  245,  246,  400,  401,  402,  403,  404,
        405,  406,  407,  408,  409,  410,  411,  412,  413,  414,  415,
        416,  417,  418,  419,  420,  421,  422,  423,  424,  425,  426,
        427,  428,  429,  430,  431,  432,  433,  434,  435,  436,  437,
        438,  439,  440,  441,  442,  443,  444,  445,  446,  447,  448,
        449,  450,  451,  452,  453,  454,  455,  456,  457,  458,  459,
        460,  461,  462,  463,  563,  564,  565,  5

In [50]:
split_indices = np.where(np.diff(array_from_index) != 1)[0] + 1

# Split the array based on the found indices
consecutive_groups = np.split(array_from_index, split_indices)

# Print the consecutive groups
for group in consecutive_groups:
    print(group)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36]
[200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 244 245 246]
[400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417
 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435
 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453
 454 455 456 457 458 459 460 461 462 463]
[563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580
 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598
 599 600 601 602]
[782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799
 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814]
[1032 1033 1034 1035 1036 1037 1038 1039 1040 1041 1042 1043 1044 1045
 1046 1047 1048 1049 1050 1051 1052 1053 1054 1055]
[1181 1182 118

In [51]:
consecutive_groups

[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36], dtype=int64),
 array([200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
        213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225,
        226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238,
        239, 240, 241, 242, 243, 244, 245, 246], dtype=int64),
 array([400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412,
        413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425,
        426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438,
        439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451,
        452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463],
       dtype=int64),
 array([563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575,
        576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588,
       

In [60]:
DucksD = player_bios.loc[(player_bios['team'] == 'ANA') & (player_bios['position'] == 'D')]
DucksDids = DucksD['playerId'].values.tolist()
DucksDids

[8475462, 8475764, 8480950, 8483490, 8482142, 8481605, 8483482, 8480001]

In [65]:
filtered_shifts_df = first_shifts_df.loc[shifts_df['playerId'].isin(DucksDids)]
filtered_shifts_df

,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,...,period,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,newStart,newEnd
118,13376971,0,00:41,00:41,None,None,6,Radko,2023020175,#111111,...,1,8475462,1,00:00,ANA,24,Anaheim Ducks,517,0,41
119,13376972,0,00:50,02:49,None,None,76,Radko,2023020175,#111111,...,1,8475462,2,01:59,ANA,24,Anaheim Ducks,517,119,169
120,13376973,0,00:49,05:40,None,None,158,Radko,2023020175,#111111,...,1,8475462,3,04:51,ANA,24,Anaheim Ducks,517,291,340
121,13377120,0,00:46,07:45,None,None,189,Radko,2023020175,#111111,...,1,8475462,4,06:59,ANA,24,Anaheim Ducks,517,419,465
122,13377121,0,00:57,10:20,None,None,258,Radko,2023020175,#111111,...,1,8475462,5,09:23,ANA,24,Anaheim Ducks,517,563,620
123,13377122,0,00:46,13:17,None,None,288,Radko,2023020175,#111111,...,1,8475462,6,12:31,ANA,24,Anaheim Ducks,517,751,797
124,13377282,0,01:10,16:34,None,None,319,Radko,2023020175,#111111,...,1,8475462,7,15:24,ANA,24,Anaheim Ducks,517,924,994
125,13377283,0,00:41,18:48,None,None,402,Radko,2023020175,#111111,...,1,8475462,8,18:07,ANA,24,Anaheim Ducks,517,1087,1128
126,13377284,0,00:36,20:00,None,None,422,Radko,2023020175,#111111,...,1,8475462,9,19:24,ANA,24,Anaheim Ducks,517,1164,1200
144,13376974,0,00:48,01:29,None,None,63,Cam,2023020175,#111111,...,1,8475764,1,00:41,ANA,24,Anaheim Ducks,517,41,89


In [66]:
for index, row in filtered_shifts_df.iterrows():
    filtered_shifts_df.at[index, 'newStart'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['startTime'].split(':'))))
    filtered_shifts_df.at[index, 'newEnd'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['endTime'].split(':'))))
    
filtered_shifts_df['newStart'] = filtered_shifts_df['newStart'].astype(int)
filtered_shifts_df['newEnd'] = filtered_shifts_df['newEnd'].astype(int)
filtered_shifts_df

,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,...,period,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,newStart,newEnd
118,13376971,0,00:41,00:41,None,None,6,Radko,2023020175,#111111,...,1,8475462,1,00:00,ANA,24,Anaheim Ducks,517,0,41
119,13376972,0,00:50,02:49,None,None,76,Radko,2023020175,#111111,...,1,8475462,2,01:59,ANA,24,Anaheim Ducks,517,119,169
120,13376973,0,00:49,05:40,None,None,158,Radko,2023020175,#111111,...,1,8475462,3,04:51,ANA,24,Anaheim Ducks,517,291,340
121,13377120,0,00:46,07:45,None,None,189,Radko,2023020175,#111111,...,1,8475462,4,06:59,ANA,24,Anaheim Ducks,517,419,465
122,13377121,0,00:57,10:20,None,None,258,Radko,2023020175,#111111,...,1,8475462,5,09:23,ANA,24,Anaheim Ducks,517,563,620
123,13377122,0,00:46,13:17,None,None,288,Radko,2023020175,#111111,...,1,8475462,6,12:31,ANA,24,Anaheim Ducks,517,751,797
124,13377282,0,01:10,16:34,None,None,319,Radko,2023020175,#111111,...,1,8475462,7,15:24,ANA,24,Anaheim Ducks,517,924,994
125,13377283,0,00:41,18:48,None,None,402,Radko,2023020175,#111111,...,1,8475462,8,18:07,ANA,24,Anaheim Ducks,517,1087,1128
126,13377284,0,00:36,20:00,None,None,422,Radko,2023020175,#111111,...,1,8475462,9,19:24,ANA,24,Anaheim Ducks,517,1164,1200
144,13376974,0,00:48,01:29,None,None,63,Cam,2023020175,#111111,...,1,8475764,1,00:41,ANA,24,Anaheim Ducks,517,41,89


In [90]:
onice = pd.DataFrame()

for i in range(1200):
    t = i+1
    active = []
    for index, row in filtered_shifts_df.iterrows():
        if (t > row['newStart']) & (t <=row['newEnd']):
            active.append(f"{row['firstName']} {row['lastName']}")
    active += [''] * (4 - len(active))
    temp = {f'p{k+1}': active[k] for k in range(4)}
    temp['second'] = t
    temp = pd.DataFrame([temp])
    onice = pd.concat([onice, temp], axis=0, ignore_index=True)
                          
onice

,p1,p2,p3,p4,second
0,Radko Gudas,Urho Vaakanainen,,,1
1,Radko Gudas,Urho Vaakanainen,,,2
2,Radko Gudas,Urho Vaakanainen,,,3
3,Radko Gudas,Urho Vaakanainen,,,4
4,Radko Gudas,Urho Vaakanainen,,,5
...,...,...,...,...,...
1195,Radko Gudas,Urho Vaakanainen,,,1196
1196,Radko Gudas,Urho Vaakanainen,,,1197
1197,Radko Gudas,Urho Vaakanainen,,,1198
1198,Radko Gudas,Urho Vaakanainen,,,1199


In [91]:
onice['pair'] = onice['p1'] + '-' + onice['p2']


pair_frequency = onice['pair'].value_counts()

# Print the result
print(pair_frequency)

Cam Fowler-Jackson LaCombe          370
Ilya Lyubushkin-Pavel Mintyukov     337
Radko Gudas-Urho Vaakanainen        312
Radko Gudas-Cam Fowler               43
Radko Gudas-Jackson LaCombe          35
Radko Gudas-Pavel Mintyukov          35
Ilya Lyubushkin-Jackson LaCombe      20
Jackson LaCombe-Pavel Mintyukov      17
Cam Fowler-Pavel Mintyukov           12
Radko Gudas-Ilya Lyubushkin          11
Urho Vaakanainen-Ilya Lyubushkin      6
Cam Fowler-Ilya Lyubushkin            2
Name: pair, dtype: int64


In [92]:
plays = requests.get(playURL).json()
plays = plays['plays']

plays = pd.DataFrame(plays)

plays

first_plays = plays.loc[plays['period'] == 1]

for index, row in first_plays.iterrows():
    first_plays.at[index, 'second'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['timeInPeriod'].split(':'))))

first_plays['second'] = first_plays['second'].astype(int)

first_plays

,eventId,period,periodDescriptor,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,details,second
0,102,1,"{'number': 1, 'periodType': 'REG'}",00:00,20:00,1551,left,520,period-start,8,NaN,0
1,101,1,"{'number': 1, 'periodType': 'REG'}",00:00,20:00,1551,left,502,faceoff,11,"{'eventOwnerTeamId': 54, 'losingPlayerId': 847...",0
2,103,1,"{'number': 1, 'periodType': 'REG'}",00:27,19:33,1551,left,525,takeaway,12,"{'xCoord': -14, 'yCoord': 39, 'zoneCode': 'N',...",27
3,65,1,"{'number': 1, 'periodType': 'REG'}",01:13,18:47,1551,left,506,shot-on-goal,23,"{'xCoord': -64, 'yCoord': 17, 'zoneCode': 'O',...",73
4,66,1,"{'number': 1, 'periodType': 'REG'}",01:23,18:37,1551,left,506,shot-on-goal,24,"{'xCoord': -67, 'yCoord': 28, 'zoneCode': 'O',...",83
...,...,...,...,...,...,...,...,...,...,...,...,...
70,212,1,"{'number': 1, 'periodType': 'REG'}",18:57,01:03,1551,left,506,shot-on-goal,257,"{'xCoord': 86, 'yCoord': 7, 'zoneCode': 'O', '...",1137
71,28,1,"{'number': 1, 'periodType': 'REG'}",19:36,00:24,1551,left,503,hit,266,"{'xCoord': 97, 'yCoord': -6, 'zoneCode': 'D', ...",1176
72,27,1,"{'number': 1, 'periodType': 'REG'}",19:39,00:21,1551,left,503,hit,267,"{'xCoord': 74, 'yCoord': 35, 'zoneCode': 'D', ...",1179
73,424,1,"{'number': 1, 'periodType': 'REG'}",19:43,00:17,1551,left,508,blocked-shot,269,"{'xCoord': 70, 'yCoord': -3, 'zoneCode': 'D', ...",1183


In [93]:
combined = pd.merge(onice, first_plays, how='left', on='second')
combined.loc[combined['typeDescKey'] == 'goal']

,p1,p2,p3,p4,second,pair,eventId,period,periodDescriptor,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,details
532,Ilya Lyubushkin,Jackson LaCombe,,,531,Ilya Lyubushkin-Jackson LaCombe,251.0,1.0,"{'number': 1, 'periodType': 'REG'}",08:51,11:09,1551,left,505.0,goal,123.0,"{'xCoord': -85, 'yCoord': -3, 'zoneCode': 'O',..."


In [94]:
onice['p3'].value_counts()

                   1198
Jackson LaCombe       1
Pavel Mintyukov       1
Name: p3, dtype: int64

In [89]:
first_shifts_df.loc[(first_shifts_df['newStart'] == 531) | (first_shifts_df['newEnd'] == 531)]

,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,...,period,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,newStart,newEnd
191,13377128,0,00:50,09:41,None,None,254,Mark,2023020175,#B9975B,...,1,8475913,3,08:51,VGK,54,Vegas Golden Knights,517,531,581
234,13377131,0,01:06,08:51,None,None,193,Ryan,2023020175,#111111,...,1,8476458,3,07:45,ANA,24,Anaheim Ducks,517,465,531
252,13377133,0,01:04,08:51,None,None,194,Jonathan,2023020175,#B9975B,...,1,8476539,3,07:47,VGK,54,Vegas Golden Knights,517,467,531
269,13377135,0,00:49,09:40,None,None,254,Chandler,2023020175,#B9975B,...,1,8476905,3,08:51,VGK,54,Vegas Golden Knights,517,531,580
333,13377142,0,00:50,09:41,None,None,254,William,2023020175,#B9975B,...,1,8477478,3,08:51,VGK,54,Vegas Golden Knights,517,531,581
357,13378048,805,None,08:51,EVG,Jack Eichel,251,Ivan,2023020175,#B9975B,...,1,8477964,0,08:51,VGK,54,Vegas Golden Knights,505,531,531
360,13377145,0,01:03,08:51,None,None,196,Ivan,2023020175,#B9975B,...,1,8477964,3,07:48,VGK,54,Vegas Golden Knights,517,468,531
390,13377149,0,01:04,08:51,None,None,195,Frank,2023020175,#111111,...,1,8478366,3,07:47,ANA,24,Anaheim Ducks,517,467,531
412,13377151,0,01:04,08:51,None,None,194,Jack,2023020175,#B9975B,...,1,8478403,3,07:47,VGK,54,Vegas Golden Knights,517,467,531
447,13377154,0,00:32,09:23,None,None,255,Troy,2023020175,#111111,...,1,8478873,3,08:51,ANA,24,Anaheim Ducks,517,531,563
